In [1]:
# Configure base path and set up Google Drive if running in Colab
if "CONFIG_DONE" not in globals():
    %run "project_path.ipynb"
else:
    print("Config done already")

Starting config...
Running in Colab?: No

base_path =  /home/mrgomez/Documents/Erdos/DL 2025/mtg-draft-agent

Check if we cd'd to the right place...
os.getcwd(): /home/mrgomez/Documents/Erdos/DL 2025/mtg-draft-agent

Configuration done


In [2]:
import zipfile
import pandas as pd

# For managing paths
import pathlib

Loading data

In [3]:
# Folder containing all the data
folder = "MTGdraft"

# Expansion code
expansion = "DSK"

In [7]:
draftfilename = "draft_data_public.DSK.PremierDraft.csv.gz"
draft_file = pathlib.Path(folder, expansion, draftfilename)

draftdata = pd.read_csv(draft_file, compression="gzip", nrows=10000)

In [8]:
gamefilename = "game_data_public.DSK.PremierDraft.csv.gz"
game_file = pathlib.Path(folder, expansion, gamefilename)

# load 'user_game_win_rate_bucket' to only import rows in the .68 bucket
gamereadindex = pd.read_csv(
    game_file, compression="gzip", usecols=["user_game_win_rate_bucket"]
)
print(gamereadindex["user_game_win_rate_bucket"] == "0.68")

0          False
1          False
2          False
3          False
4          False
           ...  
1011944    False
1011945    False
1011946    False
1011947    False
1011948    False
Name: user_game_win_rate_bucket, Length: 1011949, dtype: bool


In [9]:
# skiprows indexing starts with the row of column names. Need to add 1 to all
skipgameindexes = list(
    gamereadindex[gamereadindex["user_game_win_rate_bucket"] != 0.68].index
)
skipgameindexes = [index + 1 for index in skipgameindexes]
print(len(skipgameindexes))
gamedata = pd.read_csv(game_file, compression="gzip", skiprows=skipgameindexes)
gamedata.head()

1001897


,expansion,event_type,draft_id,draft_time,game_time,build_index,match_number,game_number,rank,opp_rank,...,tutored_Withering Torment,deck_Withering Torment,sideboard_Withering Torment,"opening_hand_Zimone, All-Questioning","drawn_Zimone, All-Questioning","tutored_Zimone, All-Questioning","deck_Zimone, All-Questioning","sideboard_Zimone, All-Questioning",user_n_games_bucket,user_game_win_rate_bucket
0,DSK,PremierDraft,1cd2f710f0d34f73ba0af31fa8cb691f,2024-09-24 18:29:43,2024-09-24 19:04:51,0,1,1,silver,NaN,...,0,0,0,0,0,0,0,0,10,0.68
1,DSK,PremierDraft,1cd2f710f0d34f73ba0af31fa8cb691f,2024-09-24 18:29:43,2024-09-24 19:22:54,0,2,1,silver,NaN,...,0,0,0,0,0,0,0,0,10,0.68
2,DSK,PremierDraft,1cd2f710f0d34f73ba0af31fa8cb691f,2024-09-24 18:29:43,2024-09-24 19:30:47,0,3,1,silver,NaN,...,0,0,0,0,0,0,0,0,10,0.68
3,DSK,PremierDraft,1cd2f710f0d34f73ba0af31fa8cb691f,2024-09-24 18:29:43,2024-09-24 19:41:44,0,4,1,silver,NaN,...,0,0,0,0,0,0,0,0,10,0.68
4,DSK,PremierDraft,4dcea824a97a4c199c0c90d6b40aba39,2024-09-30 17:33:52,2024-09-30 17:59:58,0,1,1,silver,NaN,...,0,0,0,0,0,0,0,0,10,0.68


**Renaming Basic Land Columns for them to stand out**

In [10]:
basic_lands_columns = list(
    gamedata.filter(regex="Plains$|Island$|Swamp$|Mountain$|Forest$").columns
)
# add '_Basic_Land' to these columns
str_basic = "_Basic_Land"
basic_lands_dict = {key: key + str_basic for key in basic_lands_columns}
gamedata.rename(columns=basic_lands_dict, inplace=True)

In [11]:
replayfilename = "replay_data_public.DSK.PremierDraft.csv.gz"
replay_file = pathlib.Path(folder, expansion, replayfilename)

replaydata = pd.read_csv(replay_file, compression="gzip", nrows=10000)

/tmp/ipykernel_143610/1921796635.py:4: DtypeWarning: Columns (19,20,21,26,88,91,92,93,96,97,98,100,104,107,114,115,116,117,121,122,123,126,127,128,130,136,144,146,147,152,153,154,155,156,159,160,161,162,163,166,167,169,179,182,183,184,185,187,188,190,191,192,193,196,197,198,213,214,215,216,217,218,219,221,223,224,225,228,229,230,231,244,245,247,248,249,250,253,254,255,258,259,260,275,276,277,278,279,280,281,283,285,287,290,291,292,293,306,307,309,310,311,312,315,316,320,321,322,323,337,338,339,342,343,345,347,352,354,355,368,369,371,372,373,374,377,379,382,383,384,385,399,400,401,403,404,405,407,414,415,416,417,430,431,433,434,435,436,439,444,445,446,447,461,462,463,466,467,469,476,478,479,492,493,494,495,496,497,498,501,503,506,507,508,509,523,524,525,526,527,528,529,531,533,538,539,540,541,554,555,557,558,559,560,563,565,568,569,570,571,585,586,587,589,590,591,593,595,600,601,602,603,616,617,618,619,620,621,622,623,624,625,626,627,630,631,632,633,646,647,648,649,650,651,652,653,655,6

In [12]:
list(gamedata.columns)

['expansion',
 'event_type',
 'draft_id',
 'draft_time',
 'game_time',
 'build_index',
 'match_number',
 'game_number',
 'rank',
 'opp_rank',
 'main_colors',
 'splash_colors',
 'on_play',
 'num_mulligans',
 'opp_num_mulligans',
 'opp_colors',
 'num_turns',
 'won',
 'opening_hand_Abandoned Campground',
 'drawn_Abandoned Campground',
 'tutored_Abandoned Campground',
 'deck_Abandoned Campground',
 'sideboard_Abandoned Campground',
 'opening_hand_Abhorrent Oculus',
 'drawn_Abhorrent Oculus',
 'tutored_Abhorrent Oculus',
 'deck_Abhorrent Oculus',
 'sideboard_Abhorrent Oculus',
 'opening_hand_Acrobatic Cheerleader',
 'drawn_Acrobatic Cheerleader',
 'tutored_Acrobatic Cheerleader',
 'deck_Acrobatic Cheerleader',
 'sideboard_Acrobatic Cheerleader',
 'opening_hand_Altanak, the Thrice-Called',
 'drawn_Altanak, the Thrice-Called',
 'tutored_Altanak, the Thrice-Called',
 'deck_Altanak, the Thrice-Called',
 'sideboard_Altanak, the Thrice-Called',
 'opening_hand_Anthropede',
 'drawn_Anthropede',
 't

In [13]:
# prompt: filter gamedata for the columns that start with 'deck','user', 'main_colors', etc...

deck_gamedata = gamedata.filter(
    regex="^draft_id|^build|^main_colors|^splash_colors|^won|^match_number|^deck|^user"
)

In [14]:
# prompt: remove duplicates from deck_gamedata with the same 'draft_id' and 'build_index'

deck_gamedata.drop_duplicates(
    subset=["draft_id", "build_index"], inplace=True, keep="last"
)
deck_gamedata.shape

/tmp/ipykernel_143610/1570835697.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  deck_gamedata.drop_duplicates(subset=['draft_id', 'build_index'], inplace=True, keep='last')


(1848, 294)

In [15]:
deck_gamedata.head(10)

,draft_id,build_index,match_number,main_colors,splash_colors,won,deck_Abandoned Campground,deck_Abhorrent Oculus,deck_Acrobatic Cheerleader,"deck_Altanak, the Thrice-Called",...,deck_Waltz of Rage,deck_Wary Watchdog,deck_Wickerfolk Thresher,deck_Wildfire Wickerfolk,deck_Winter's Intervention,"deck_Winter, Misanthropic Guide",deck_Withering Torment,"deck_Zimone, All-Questioning",user_n_games_bucket,user_game_win_rate_bucket
3,1cd2f710f0d34f73ba0af31fa8cb691f,0,4,UG,R,False,0,0,0,0,...,0,1,0,0,0,0,0,0,10,0.68
4,4dcea824a97a4c199c0c90d6b40aba39,0,1,WR,B,True,0,0,0,0,...,0,0,0,0,0,0,0,0,10,0.68
12,4dcea824a97a4c199c0c90d6b40aba39,1,9,WR,B,False,0,0,0,0,...,0,0,0,0,0,0,0,0,10,0.68
15,5f6e7fc602474128b0a3a0078fc54958,0,3,UB,NaN,False,0,0,0,0,...,0,0,0,0,0,0,0,0,10,0.68
21,5f6e7fc602474128b0a3a0078fc54958,1,9,UB,NaN,True,0,0,0,0,...,0,0,0,0,0,0,0,0,10,0.68
29,fbd40749f6fe4c98853f99e5ec520b3a,0,8,WR,NaN,True,0,0,0,0,...,0,0,0,0,0,0,0,0,10,0.68
37,131f24fec41f4848bc9a8d13496775e8,0,8,WR,NaN,False,0,0,1,0,...,0,0,0,0,0,0,0,0,10,0.68
44,9956da2ee6174cba92f84d576f7932b2,0,7,BG,WU,False,0,0,0,0,...,0,0,0,0,0,0,1,0,10,0.68
52,5b968712d9de4cabb55453aaa321b61c,0,8,WR,U,True,0,0,1,0,...,0,0,0,0,0,0,0,0,10,0.68
61,663a88f4cd4e40adb05aad9951a115ad,0,9,UG,NaN,True,0,0,0,0,...,0,0,0,0,0,0,0,1,10,0.68


We start by doing a PCA of the deck data. We want to make an interactive plot where we can check the each data point when we hover over it.

We will first make a function that extracts the decklist from a deck vector.

In [16]:
# utility: this extracts decklists
# prompt: make a function whose inputs are the 'draft_id' and 'build_number' and outputs a table corresponding to column [ 'draft_id', 'build_number'] of deck_vector_trans with the name of each non_zero row together with its corresponding number


def get_decklist(draft_id, build_number, deck_gamedata):
    """
    Extracts the non-zero card counts for a specific draft_id and build_number.

    Args:
        draft_id (str or int): The ID of the draft.
        build_number (int): The build number within the draft.
        deck_vector_transposed (pd.DataFrame): The transposed DataFrame with
                                                (draft_id, build_index) as columns
                                                and card names as index.

    Returns:
        pd.DataFrame: A DataFrame with card names and their counts for the
                      specified draft_id and build_number, or an empty DataFrame
                      if the combination is not found.
    """

    deck_vector_transposed = (
        deck_gamedata.filter(regex="^draft_id|^build_index|^deck_")
        .set_index(["draft_id", "build_index"])
        .T
    )
    try:
        # Select the column corresponding to the given draft_id and build_number
        deck_series = deck_vector_transposed[(draft_id, build_number)]

        # Filter for non-zero values
        non_zero_cards = deck_series[deck_series > 0]

        # Create a DataFrame from the non-zero values
        decklist_df = non_zero_cards.reset_index()
        decklist_df.columns = ["Card Name", "Count"]

        return decklist_df

    except KeyError:
        print(f"No data found for draft_id: {draft_id}, build_number: {build_number}")
        return pd.DataFrame(columns=["Card Name", "Count"])


# Example usage:
# Assuming 'deck_vector_gamedata_transposed' is the DataFrame from the preceding code
# Find an example draft_id and build_index from deck_vector_gamedata_transposed.columns
# You can inspect the columns to find a valid pair
if not deck_gamedata.empty:
    example_draft_id, example_build_number = deck_gamedata[
        ["draft_id", "build_index"]
    ].iloc[0]
    decklist_table = get_decklist(example_draft_id, example_build_number, deck_gamedata)
    print(
        f"Decklist for Draft ID: {example_draft_id}, Build Number: {example_build_number}"
    )
    print(decklist_table)
else:
    print("deck_vector_gamedata_transposed is empty. Cannot get decklist.")

# You can test with other draft_id and build_number pairs found in your data
# For instance, you could pick another pair from deck_vector_gamedata.head()
# example_draft_id_2 = '...'
# example_build_number_2 = 0 # or 1
# decklist_table_2 = get_decklist(example_draft_id_2, example_build_number_2, deck_vector_gamedata_transposed)
# print(f"\nDecklist for Draft ID: {example_draft_id_2}, Build Number: {example_build_number_2}")
# print(decklist_table_2)

Decklist for Draft ID: 1cd2f710f0d34f73ba0af31fa8cb691f, Build Number: 0
                                   Card Name  Count
0                       deck_Bashful Beastie      2
1                        deck_Bleeding Woods      1
2                     deck_Cautious Survivor      2
3                     deck_Cryptid Inspector      2
4                      deck_Defiant Survivor      1
5                      deck_Enter the Enigma      1
6                        deck_Flesh Burrower      2
7                     deck_Forest_Basic_Land      8
8                    deck_Impossible Inferno      2
9                     deck_Island_Basic_Land      8
10                       deck_Lakeside Shack      1
11                       deck_Manifest Dread      2
12                  deck_Monstrous Emergence      2
13                  deck_Mountain_Basic_Land      1
14                   deck_Oblivious Bookworm      2
15            deck_Overlord of the Floodpits      1
16           deck_Overlord of the Hauntwood

In [17]:
# prompt: make a function taking the output of get_decklist and trasforming it into plain text


def decklist_to_list(decklist_df):
    """
    Transforms a decklist DataFrame into a plain text string.

    Args:
        decklist_df (pd.DataFrame): DataFrame with 'Card Name' and 'Count' columns.

    Returns:
        str: A plain text representation of the decklist.
    """
    if decklist_df.empty:
        return "Empty Decklist"

    non_basic_land_decklist = decklist_df[
        ~decklist_df["Card Name"].str.endswith("_Basic_Land")
    ]
    basic_land_decklist = decklist_df[
        decklist_df["Card Name"].str.endswith("_Basic_Land")
    ]

    lines = [
        f"{row['Count']} {row['Card Name']}"
        for index, row in non_basic_land_decklist.iterrows()
    ] + [
        f"{row['Count']} {row['Card Name']}"
        for index, row in basic_land_decklist.iterrows()
    ]
    return lines


# Example usage with the previously obtained decklist_table:
list_decklist = decklist_to_list(
    get_decklist("1936edc36c834030a9d3a6d566059c66", 0, deck_gamedata)
)
print("List decklist:")
print(list_decklist)

List decklist:
['2 deck_Bottomless Pool // Locker Room', '1 deck_Creeping Peeper', '2 deck_Cursed Windbreaker', '1 deck_Daggermaw Megalodon', "1 deck_Don't Make a Sound", '1 deck_Fear of Impostors', '1 deck_Floodpits Drowner', '1 deck_Get Out', '2 deck_Glimmerburst', '1 deck_Meat Locker // Drowned Diner', '1 deck_Overlord of the Floodpits', '1 deck_Piranha Fly', '2 deck_Stalked Researcher', '1 deck_Tunnel Surveyor', '3 deck_Twist Reality', '1 deck_Unnerving Grasp', '2 deck_Vanish from Sight', '16 deck_Island_Basic_Land']


Test Data

In [18]:
dataf = pd.DataFrame(
    {"name": ["deck1", "deck2"], "dkq": [plaintext_decklist, plaintext_decklist]}
)
dataf

NameError: name 'plaintext_decklist' is not defined

altair test

In [19]:
test_chart = (
    alt.Chart(dataf)
    .mark_point()
    .encode(
        x="name:N",
        y="deck:N",
    )
    .transform_calculate("deck", "split(datum['dkq'], '@')")
)

# Display the chart
test_chart

NameError: name 'alt' is not defined

In [ ]:
def decklist_list_from_row(row):
    return decklist_to_list(
        get_decklist(row["draft_id"], row["build_index"], deck_gamedata)
    )


print(decklist_list_from_row(deck_gamedata.iloc[0]))

['2 deck_Bashful Beastie', '1 deck_Bleeding Woods', '2 deck_Cautious Survivor', '2 deck_Cryptid Inspector', '1 deck_Defiant Survivor', '1 deck_Enter the Enigma', '2 deck_Flesh Burrower', '2 deck_Impossible Inferno', '1 deck_Lakeside Shack', '2 deck_Manifest Dread', '2 deck_Monstrous Emergence', '2 deck_Oblivious Bookworm', '1 deck_Overlord of the Floodpits', '1 deck_Overlord of the Hauntwoods', '1 deck_Peculiar Lighthouse', '1 deck_Piranha Fly', '1 deck_Rootwise Survivor', '1 deck_Slavering Branchsnapper', '1 deck_Twitching Doll', '3 deck_Underwater Tunnel // Slimy Aquarium', '1 deck_Vanish from Sight', '1 deck_Wary Watchdog', '8 deck_Forest_Basic_Land', '8 deck_Island_Basic_Land', '1 deck_Mountain_Basic_Land']


In [ ]:
# prompt: plot pca of deck_gamedata to 2 dimensions

from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import numpy as np

# Assuming deck_gamedata contains numerical features suitable for PCA
# We need to select the columns to use for PCA, excluding non-numerical ones
# Let's drop 'draft_id', 'main_colors', 'splash_colors', 'deck_id', 'user_id', 'build_index'
# We should also handle potential NaNs if they exist in the selected columns
columns_to_drop = [
    "draft_id",
    "main_colors",
    "match_number",
    "splash_colors",
    "won",
    "build_index",
    "user_n_games_bucket",
    "user_game_win_rate_bucket",
] + [key for key in deck_gamedata.columns if key.endswith("_Basic_Land")]
pca_data = deck_gamedata.drop(columns=columns_to_drop)


# Handle missing values by imputing with the mean (a simple approach)
pca_data = pca_data.fillna(0)

# Perform PCA
pca = PCA(n_components=2)
principal_components = pca.fit_transform(pca_data)

# Create a DataFrame with the principal components
pca_df = pd.DataFrame(
    data=principal_components,
    columns=["principal_component_1", "principal_component_2"],
)

# Assuming pca_df, deck_gamedata are already defined and processed from the preceding code

# Add draft_id and build_index to the pca_df for interactive selection
# Ensure that the order of rows in pca_df matches the order in deck_gamedata
pca_df["draft_id"] = deck_gamedata["draft_id"].values
pca_df["build_index"] = deck_gamedata["build_index"].values
pca_df["main_colors"] = deck_gamedata["main_colors"].values
pca_df["splash_colors"] = deck_gamedata["splash_colors"].values
pca_df["decklist"] = pca_df.apply(decklist_list_from_row, axis=1)
pca_df["won"] = deck_gamedata["won"].values
property_lists = [
    [
        "" + str(key) + ": " + str(row[key])
        for key in row.keys()
        if not key.endswith("decklist")
    ]
    + ["decklist:"]
    + row["decklist"]
    for index, row in pca_df.iterrows()
]
pca_df["properties"] = property_lists
print(pca_df.head())

   principal_component_1  principal_component_2  \
0              -2.526207              -1.432442   
1               1.719393              -0.050446   
2               1.696952               0.019764   
3              -0.821696               0.626451   
4              -0.786263               0.505038   

                           draft_id  build_index main_colors splash_colors  \
0  1cd2f710f0d34f73ba0af31fa8cb691f            0          UG             R   
1  4dcea824a97a4c199c0c90d6b40aba39            0          WR             B   
2  4dcea824a97a4c199c0c90d6b40aba39            1          WR             B   
3  5f6e7fc602474128b0a3a0078fc54958            0          UB           NaN   
4  5f6e7fc602474128b0a3a0078fc54958            1          UB           NaN   

                                            decklist    won  \
0  [2 deck_Bashful Beastie, 1 deck_Bleeding Woods...  False   
1  [2 deck_Boilerbilges Ripper, 2 deck_Clockwork ...   True   
2  [1 deck_Boilerbilges Ripper, 2 d

**Dictionary for Altair Colors**

In [ ]:
# prompt: plot pca of deck_gamedata to 2 dimensions for each 'main_colors'
colorlist = deck_gamedata["main_colors"].unique()
print(colorlist)
color_list_dict = {
    "W": ["white"],
    "U": ["blue"],
    "B": ["black"],
    "R": ["red"],
    "G": ["green"],
    "WU": ["white", "blue"],
    "WB": ["white", "black"],
    "WR": ["white", "red"],
    "WG": ["white", "green"],
    "UB": ["blue", "black"],
    "UR": ["blue", "red"],
    "UG": ["blue", "green"],
    "BR": ["black", "red"],
    "BG": ["black", "green"],
    "RG": ["red", "green"],
    "WUB": ["white", "blue", "black"],
    "WUR": ["white", "blue", "red"],
    "WUG": ["white", "blue", "green}"],
    "WBR": ["white", "black", "red"],
    "WBG": ["white", "black", "green"],
    "WRG": ["white", "red", "green"],
    "UBR": ["blue", "black", "red"],
    "UBG": ["blue", "black", "green"],
    "URG": ["blue", "red", "green"],
    "BRG": ["black", "red", "green"],
    "WUBR": ["white", "blue", "black", "red"],
    "WUBG": ["white", "blue", "black", "green"],
    "WURG": ["white", "blue", "red", "green"],
    "WBRG": ["white", "black", "red", "green"],
    "UBRG": ["blue", "black", "red", "green"],
    "WUBRG": ["white", "blue", "black", "red", "green"],
}

['UG' 'WR' 'UB' 'BG' 'WU' 'RG' 'WB' 'G' 'BR' 'WUB' 'WBRG' 'BRG' 'URG'
 'WUG' 'R' 'WBG' 'WUR' 'WG' 'UR' 'UBG' 'WBR' 'U' 'B' 'WUBRG' 'WRG' 'UBRG'
 'W' 'UBR']


In [ ]:
averaged_colors_dict = {
    "": "#888888",
    "W": "#f8e4a0",
    "U": "#0000ff",
    "B": "#000000",
    "R": "#ff0000",
    "G": "#00ff00",
    "WU": "#7c72cf",
    "WB": "#7c7250",
    "WR": "#fb7250",
    "WG": "#7cf150",
    "UB": "#00007f",
    "UR": "#7f007f",
    "UG": "#007f7f",
    "BR": "#7f0000",
    "BG": "#007f00",
    "RG": "#7f7f00",
    "WUB": "#524c8a",
    "WUR": "#a74c8a",
    "WUG": "#7c72cf",
    "WBR": "#a74c35",
    "WBG": "#52a135",
    "WRG": "#a7a135",
    "UBR": "#550055",
    "UBG": "#005555",
    "URG": "#555555",
    "BRG": "#555500",
    "WUBR": "#7d3967",
    "WUBG": "#3e7867",
    "WURG": "#7d7867",
    "WBRG": "#7d7828",
    "UBRG": "#3f3f3f",
    "WUBRG": "#646053",
}

In [ ]:
list1 = [tuple(sorted(key)) for key in averaged_colors_dict]
sorted(list1[30])

['B', 'G', 'R', 'U']

In [ ]:
sorted("CVFDSasdfgh")

['C', 'D', 'F', 'S', 'V', 'a', 'd', 'f', 'g', 'h', 's']

**Task: Create Altair Chart with PCcomps , 'draft'id', 'build_index', etc**

In [ ]:
# prompt: make the previous chart interactive, showing the get_decklist(draft_id, build_number, deck_gamedata) of the data point we hover the cursor over

# Update the tooltip encoding
tooltip = [
    alt.Tooltip("principal_component_1:Q", title="PC1"),
    alt.Tooltip("principal_component_2:Q", title="PC2"),
    alt.Tooltip("won:N", title="Won"),
    alt.Tooltip("draft_id:N", title="Draft ID"),
    alt.Tooltip("build_index:Q", title="Build Index"),
    alt.Tooltip("main_colors:N", title="Main Colors"),
    alt.Tooltip("splash_colors:N", title="Splash Colors"),
    alt.Tooltip("decklist:N", title="Decklist"),
    alt.Tooltip("properties:N", title="Properties"),
]

# Assuming averaged_colors_dict is already defined. This defines the colors for each color combination
custom_scale = alt.Scale(
    domain=list(averaged_colors_dict.keys()), range=list(averaged_colors_dict.values())
)

# Recreate the Altair chart with the updated tooltip
chart = (
    alt.Chart(pca_df)
    .mark_point()
    .encode(
        x="principal_component_1:Q",
        y="principal_component_2:Q",
        color=alt.Color(
            "main_colors", scale=custom_scale, legend=alt.Legend(orient="left")
        ),  # Color based on 'main_colors'
        shape=alt.Shape(
            "won", legend=alt.Legend(orient="bottom")
        ),  # Shape based on 'main_colors'
        tooltip=tooltip,  # Use the updated tooltip list
    )
    .properties(title=["2D PCA of Deck Gamedata", "(Interactive)"])
    .interactive()
)  # Make the chart interactive

# Display the chart
chart

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# prompt: add a separate chart on the right showing the currently selected point 'properties'

import altair as alt

# Add a selection layer for interactivity
selection = alt.selection_point(
    fields=["principal_component_1", "principal_component_2"],
    name="point",
    nearest=True,
)

# Assuming averaged_colors_dict is already defined. This defines the colors for each color combination
custom_scale = alt.Scale(
    domain=list(averaged_colors_dict.keys()), range=list(averaged_colors_dict.values())
)

# Update the tooltip encoding
tooltip = [
    alt.Tooltip("principal_component_1:Q", title="PC1"),
    alt.Tooltip("principal_component_2:Q", title="PC2"),
    alt.Tooltip("won:N", title="Won"),
    alt.Tooltip("draft_id:N", title="Draft ID"),
    alt.Tooltip("build_index:Q", title="Build Index"),
    alt.Tooltip("main_colors:N", title="Main Colors"),
    alt.Tooltip("splash_colors:N", title="Splash Colors"),
]

# Create the base chart for the scatter plot
scatter_chart = (
    alt.Chart(pca_df)
    .mark_point()
    .encode(
        x="principal_component_1:Q",
        y="principal_component_2:Q",
        color=alt.Color(
            "main_colors", scale=custom_scale, legend=alt.Legend(orient="left")
        ),  # Color based on 'main_colors'
        shape=alt.Shape(
            "won", legend=alt.Legend(orient="bottom")
        ),  # Shape based on 'main_colors'
        tooltip=tooltip,  # Use the updated tooltip list
        opacity=alt.condition(
            selection, alt.value(1), alt.value(0.2)
        ),  # Make points transparent when not selected
    )
    .properties(title="2D PCA of Deck Gamedata (Interactive)")
    .add_params(selection)
    .interactive()
)  # Make the scatter chart interactive

# Create the chart for the selected point's 'properties'
# This chart will only show data for the selected point due to the filter
selected_prop = (
    alt.Chart(pca_df)
    .mark_text(align="left")
    .encode(  # Added .mark_text()
        text="properties:N",  # Display the 'main_colors'
    )
    .transform_filter(
        selection  # Filter this chart based on the selection from the scatter chart
    )
    .properties(title="Selected Properties")  # Title for this chart
)

# Combine the two charts side by side
combined_chart = scatter_chart | selected_prop

# Display the combined chart
combined_chart

Output hidden; open in https://colab.research.google.com to view.

Function for PCA processing

In [ ]:
def pca_processing_gamedata(deck_gamedata):
    # Assuming deck_gamedata contains numerical features suitable for PCA
    # We need to select the columns to use for PCA, excluding non-numerical ones
    # Let's drop 'draft_id', 'main_colors', 'splash_colors', 'deck_id', 'user_id', 'build_index'
    # We should also handle potential NaNs if they exist in the selected columns
    columns_to_drop = [
        "draft_id",
        "main_colors",
        "match_number",
        "splash_colors",
        "won",
        "build_index",
        "user_n_games_bucket",
        "user_game_win_rate_bucket",
    ] + [key for key in deck_gamedata.columns if key.endswith("_Basic_Land")]
    pca_data = deck_gamedata.drop(columns=columns_to_drop)

    # Handle missing values by imputing with the mean (a simple approach)
    pca_data = pca_data.fillna(0)

    # Perform PCA
    pca = PCA(n_components=2)
    principal_components = pca.fit_transform(pca_data)

    # Create a DataFrame with the principal components
    pca_df = pd.DataFrame(
        data=principal_components,
        columns=["principal_component_1", "principal_component_2"],
    )

    # Assuming pca_df, deck_gamedata are already defined and processed from the preceding code

    # Add draft_id and build_index to the pca_df for interactive selection
    # Ensure that the order of rows in pca_df matches the order in deck_gamedata
    pca_df["draft_id"] = deck_gamedata["draft_id"].values
    pca_df["build_index"] = deck_gamedata["build_index"].values
    pca_df["main_colors"] = deck_gamedata["main_colors"].values
    pca_df["splash_colors"] = deck_gamedata["splash_colors"].values
    pca_df["decklist"] = pca_df.apply(decklist_list_from_row, axis=1)
    pca_df["won"] = deck_gamedata["won"].values
    property_lists = [
        [
            "" + str(key) + ": " + str(row[key])
            for key in row.keys()
            if not key.endswith("decklist")
        ]
        + ["decklist:"]
        + row["decklist"]
        for index, row in pca_df.iterrows()
    ]
    pca_df["properties"] = property_lists
    return pca_df

Function to create chart from pca data

In [ ]:
# function to create charts
def create_chart_from_pca(pca_df):
    # Add a selection layer for interactivity
    selection = alt.selection_point(
        fields=["principal_component_1", "principal_component_2"],
        name="point",
        nearest=True,
    )

    # Assuming averaged_colors_dict is already defined. This defines the colors for each color combination
    custom_scale = alt.Scale(
        domain=list(averaged_colors_dict.keys()),
        range=list(averaged_colors_dict.values()),
    )

    # Update the tooltip encoding
    tooltip = [
        alt.Tooltip("principal_component_1:Q", title="PC1"),
        alt.Tooltip("principal_component_2:Q", title="PC2"),
        alt.Tooltip("won:N", title="Won"),
        alt.Tooltip("draft_id:N", title="Draft ID"),
        alt.Tooltip("build_index:Q", title="Build Index"),
        alt.Tooltip("main_colors:N", title="Main Colors"),
        alt.Tooltip("splash_colors:N", title="Splash Colors"),
    ]

    # Create the base chart for the scatter plot
    scatter_chart = (
        alt.Chart(pca_df)
        .mark_point()
        .encode(
            x="principal_component_1:Q",
            y="principal_component_2:Q",
            color=alt.Color(
                "main_colors", scale=custom_scale, legend=alt.Legend(orient="left")
            ),  # Color based on 'main_colors'
            shape=alt.Shape(
                "won", legend=alt.Legend(orient="bottom")
            ),  # Shape based on 'main_colors'
            tooltip=tooltip,  # Use the updated tooltip list
            opacity=alt.condition(
                selection, alt.value(1), alt.value(0.2)
            ),  # Make points transparent when not selected
        )
        .properties(title="2D PCA of Deck Gamedata (Interactive)")
        .add_params(selection)
        .interactive()
    )  # Make the scatter chart interactive

    # Create the chart for the selected point's 'properties'
    # This chart will only show data for the selected point due to the filter
    selected_prop = (
        alt.Chart(pca_df)
        .mark_text(align="left")
        .encode(  # Added .mark_text()
            text="properties:N",  # Display the 'main_colors'
        )
        .transform_filter(
            selection  # Filter this chart based on the selection from the scatter chart
        )
        .properties(title="Selected Properties")  # Title for this chart
    )

    # Combine the two charts side by side
    combined_chart = scatter_chart | selected_prop

    # Display the combined chart
    return combined_chart


# test reate_chart_from_pca (pca_df)
create_chart_from_pca(pca_df)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
from enum import unique

# prompt: plot pca of deck_gamedata to 2 dimensions for each 'main_colors'
colorlist = deck_gamedata["main_colors"].unique()
print(colorlist)

['UG' 'WR' 'UB' 'BG' 'WU' 'RG' 'WB' 'G' 'BR' 'WUB' 'WBRG' 'BRG' 'URG'
 'WUG' 'R' 'WBG' 'WUR' 'WG' 'UR' 'UBG' 'WBR' 'U' 'B' 'WUBRG' 'WRG' 'UBRG'
 'W' 'UBR']


In [ ]:
# now do a pca for each color
pca_data_dict = {}
for color in colorlist:
    print(color)
    pca_data_dict[color] = deck_gamedata[deck_gamedata["main_colors"] == color]
    if len(pca_data_dict[color]) > 3:
        pca_data_dict[color] = pca_processing_gamedata(pca_data_dict[color])
        color_chart = create_chart_from_pca(pca_data_dict[color])
        display(color_chart)

Output hidden; open in https://colab.research.google.com to view.